In [1]:
import numpy as np
import pandas as pd
import graphtools
import networkx as nx
import scanpy, phate, scprep
import matplotlib.pyplot as plt
import seaborn as sns
import magic
from sklearn.preprocessing import scale

In [4]:
df = pd.read_csv('results/gene_summary.csv', index_col=0)
results = np.load('results/GSPA_QR_without_regression.npz', allow_pickle=True)
df = df.sort_values(['gene_clusters', 'localization_score'], ascending=[True, False])

In [6]:
for cluster in range(6):
    threshold = np.percentile(df[df['gene_clusters'] == cluster]['localization_score'], q=90)
    loc_genes = df[(df['gene_clusters'] == cluster) & (df['localization_score'] > threshold)].index
    loc_emb = pd.DataFrame(results['signal_embedding'], index=results['genes'])
    loc_emb = loc_emb.loc[loc_genes]
    
    G = nx.Graph(graphtools.Graph(loc_emb, use_pygsp=True, knn=5).A)
    G = nx.relabel_nodes(G, dict(zip(range(len(loc_emb)), loc_genes)))
    nx.write_edgelist(G, f'results/Cluster_{cluster}_loc_genes.tsv', delimiter='\t', data=False)